In [1]:
import numpy as np
import keras
from tensorflow.keras import layers

num_words = 10000  # only keep top 10k words

(train_data, train_labels), (test_data, test_labels) = keras.datasets.reuters.load_data(
    num_words=num_words
)

print("Training samples:", len(train_data))
print("Test samples:", len(test_data))
print("Example sequence:", train_data[0][:10])
print("Label:", train_labels[0])

def vectorize_sequences(sequences, dimension=num_words):
    results = np.zeros((len(sequences), dimension), dtype="float32")
    for i, seq in enumerate(sequences):
        results[i, seq] = 1.0
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
num_classes = np.max(train_labels) + 1
print("Number of classes:", num_classes)

def to_one_hot(labels, dimension=num_classes):
    results = np.zeros((len(labels), dimension), dtype="float32")
    for i, label in enumerate(labels):
        results[i, label] = 1.0
    return results

y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

model = keras.Sequential([
    layers.Dense(128, activation="relu", input_shape=(num_words,)),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(128, activation="relu"),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(num_classes, activation="softmax"),
])

model.summary()

model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = y_train[:1000]
partial_y_train = y_train[1000:]

history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=10,
    batch_size=64,
    validation_data=(x_val, y_val),
    verbose=1
)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

predictions = model.predict(x_test[:5])

# Each prediction is a 46-dim probability vector
for i, p in enumerate(predictions):
    predicted_class = np.argmax(p)
    true_class = np.argmax(y_test[i])
    print(f"Sample {i}: predicted={predicted_class}, true={true_class}")



2025-12-08 23:27:50.470282: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-08 23:27:50.528772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-08 23:27:52.897357: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Training samples: 8982
Test samples: 2246
Example sequence: [1, 2, 2, 8, 43, 10, 447, 5, 25, 207]
Label: 3
x_train shape: (8982, 10000)
x_test shape: (2246, 10000)
Number of classes: 46


/home/aj/CICIDS/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-12-08 23:27:56.052289: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 46)             │         2,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

122/125 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4908 - loss: 2.1861

2025-12-08 23:28:00.093885: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 40000000 exceeds 10% of free system memory.


125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.6111 - loss: 1.6689 - val_accuracy: 0.6980 - val_loss: 1.2413
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7631 - loss: 1.0000 - val_accuracy: 0.7610 - val_loss: 1.0361
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8355 - loss: 0.6894 - val_accuracy: 0.7910 - val_loss: 0.9780
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8812 - loss: 0.4885 - val_accuracy: 0.8080 - val_loss: 1.0723
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9085 - loss: 0.3757 - val_accuracy: 0.7940 - val_loss: 1.0897
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9295 - loss: 0.2847 - val_accuracy: 0.8090 - val_loss: 1.1989
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9414 - loss: 0.2415 - val_accuracy: 0.8090 - val_loss: 1.2612
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9460 - loss: 0.2076 - val_accuracy: 0.809

2025-12-08 23:28:20.063687: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 89840000 exceeds 10% of free system memory.


Test accuracy: 0.7734
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Sample 0: predicted=3, true=3
Sample 1: predicted=2, true=10
Sample 2: predicted=1, true=1
Sample 3: predicted=4, true=4
Sample 4: predicted=13, true=4
